 ## XFOIL analysis with python UI
 
 #### we set the control point has 7 on top edge, 7 on the bottom
 
 below is the index that has to be fixed, when calling the mutation function, those two points has fixed x aixs
     
     index_fix = [6, 7]
 
 ### index

* [init](#init): initial and refresh the program
* [Bezier curve](#bezier curve): the function that builds airfoil using control points using bezier curves
* [Generator](#generator): this is to generate new airfoil using reproduction and mutation
* [File and Data Mining](#mining): this is to mine the data from files
* [Heuristic Function](#eva): the algorithm of evaluation function
* [Script](#script): couple xfoil and compose the script for calculation
* [Run](#run): execute

---
<a id='init'></a>
## init

In [287]:
import subprocess as sp
import shutil
import sys
import string
import shutil
from os.path import isfile, join
from os import listdir
import random

In [288]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def data_check(cl):
    if max(cl) < 1.4:
        return True
    else:
        return False
    
def frange(start, finish, interval=1.0):
    #frange starts from small angle to large angle
    assert finish > start, 'frange start with smaller number'
    x = float(start)
    out = [x]
    while x < float(finish):
        x+=interval
        out.append(round(x, 5))
    #make sure that the last element is the finish value
    out = list(set(out[:-1] + [float(finish)]))
    #list element is from small to large
    return sorted(out)


In [289]:
import os
from pathlib import Path
import sched, time
def boostup(data_loc, airfoil_loc):
    '''this is to restart the program by erasing all the files and datas from previous calculation'''
    p = Path(data_loc)
    #if the directory is there, delete it
    if p.is_dir():
        shutil.rmtree(data_loc)
    #when the directory is still ther, sleep
    while p.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(data_loc)
    
    #get the list of dir in a path
    f = Path(airfoil_loc)
    #if the directory is there, delete it
    if f.is_dir():
        shutil.rmtree(airfoil_loc)
    #when the directory is still ther, sleep
    while f.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(airfoil_loc)
    pass

In [290]:
def ctlPts_setup(n_foil):
    foils = {str(n): ctlPts_generator() for n in range(n_foil)}
    #foils = {foil index: control points setup}
    return foils
    
def ctlPts_generator():
    '''generate control points and save the points in backup file'''
    #set an original airfoil then do some mutation
    ctlPts = [[[1.0, 0.0],
              [0.89333, 0.12],
              [0.66667, 0.13],
              [0.5, 0.17],
              [0.33333, 0.22],
              [0.16667, 0.13],
              [0.0,0.055]],
              [[0.0, -0.05],
              [0.16667, -0.1],
              [0.33333, -0.15],
              [0.5, -0.11],
              [0.66667, -0.05],
              [0.83333, -0.031728789055187],
              [1.0, 0.0]]]
    
    ctlPts = mutation(ctlPts, 1)
    
    return ctlPts

---
<a id = 'bezier curve'></a>
## Bezier Curve

why bezier curve?
Since when we mutate cordinate file directly, sometimes it will causes crashes of xfoil, therefore using control points approach will have a nice airfoil no matter what

In [291]:
import numpy as np

#--- MAIN ---------------------------------------------------------------------+
def airfoil(loc, ctlPts, names, numPts, boo=True):
    '''this function is using bezier curve to produce airfoil from control points
    intput:   
        foil_locs: {airfoil name: cordinate location} 
        foil_ctlPts: {airfoil name: control points}
        foil_names: [airfoil names]
        numPts: number of points per bezier curve
        boo: should the file be saved (stupid parameter)
        
    '''
    ctlPts = ctlPts[0] + ctlPts[1]

    def quadraticBezier(t, points):
        #recreated the bezier curve formula
        B_x=(1-t)*((1-t)*points[0][0]+t*points[1][0])+t*((1-t)*points[1][0]+t*points[2][0])
        B_y=(1-t)*((1-t)*points[0][1]+t*points[1][1])+t*((1-t)*points[1][1]+t*points[2][1])
        return B_x,B_y

    curve=[]
    #construct the independent variable
    t=np.array([i*1/numPts for i in range(0,numPts)])

    # calculate first Bezier curve
    midX=(ctlPts[1][0]+ctlPts[2][0])/2
    midY=(ctlPts[1][1]+ctlPts[2][1])/2
    B_x0,B_y0=quadraticBezier(t,[ctlPts[0],ctlPts[1],[midX,midY]])
    curve=curve+list(zip(B_x0,B_y0))

    # calculate middle Bezier Curves
    for i in range(1,len(ctlPts)-3):
        p0=ctlPts[i]
        p1=ctlPts[i+1]
        p2=ctlPts[i+2]
        midX_1=(ctlPts[i][0]+ctlPts[i+1][0])/2
        midY_1=(ctlPts[i][1]+ctlPts[i+1][1])/2
        midX_2=(ctlPts[i+1][0]+ctlPts[i+2][0])/2
        midY_2=(ctlPts[i+1][1]+ctlPts[i+2][1])/2

        B_xi,B_yi=quadraticBezier(t,[[midX_1,midY_1],ctlPts[i+1],[midX_2,midY_2]])
        curve=curve+list(zip(B_xi,B_yi))

    # calculate last Bezier curve
    midX=(ctlPts[-3][0]+ctlPts[-2][0])/2
    midY=(ctlPts[-3][1]+ctlPts[-2][1])/2

    B_x1,B_y1=quadraticBezier(t,[[midX,midY],ctlPts[-2],ctlPts[-1]])
    curve=curve+list(zip(B_x1,B_y1))
    curve.append(ctlPts[-1])

    # write airfoil coordinates to text file
    if boo:
        xPts,yPts=zip(*curve)
        f=open(loc,'w+')
        for i in range(len(xPts)):
            f.write(str(xPts[i])+','+str(yPts[i])+'\n')
        f.close()
    
    pass
    

---
<a id='generator'></a>
## Generator

In [292]:
import itertools as it
def generator(loc, parents_ctlPts, mut_inten, gen):
    '''
    input:
        loc: the dir that where we save the airfoil cordinates
        parents_ctlPts: {airfoil name: cordinates}
        mut_inten = mutation intensity
        gen = which generation
    output:
        foil_locs: {airfoil name: cordinate location} 
        foil_ctlPts: {airfoil name: control points}
        foil_names: [airfoil names]
    '''
    #gen name
    surname = 'G'+str(gen).zfill(2)
    #airfoil name
    n_foil = 0
    first_name = 'A{}'.format
    
    foil_ctlPts = {}
    foil_locs = {}
    foil_names = []
    
    #generate all the possible combinations
    couples = list(it.combinations(list(parents_ctlPts.keys()), 2))
    for father, mother in couples:
        #papa and mama produce some children
        try:
            child1, child2 = reproduction(parents_ctlPts[father], parents_ctlPts[mother], mut_inten)
        except:
            print(father, mother)

        #get airfoil serial number and put them into dictionary
        n_foil+=1
        name = surname+first_name(n_foil)
        foil_names.append(name)
        Floc = loc+name+'.txt'
        foil_locs[name] = Floc
        foil_ctlPts[name] = child1
                
        n_foil+=1
        name = surname+first_name(n_foil)
        foil_names.append(name)
        Floc = loc+name+'.txt'
        foil_locs[name] = Floc
        foil_ctlPts[name] = child2
    
    for name in foil_names:
        ctlPts = foil_ctlPts[name]
        loc = foil_locs[name]
        #build airfoil cordinate using bezier curve
        airfoil(loc, ctlPts, name, 20)
    
    return foil_locs, foil_ctlPts, foil_names

def reproduction(father, mother, mutation_intensity):
    

    #split genes
    genesUp = [father[0], mother[0]]
    genesDown = [father[1], mother[1]]
    
    #new airfoil
    child1 = [genesUp[0], genesDown[1]]
    child2 = [genesUp[1], genesDown[0]]
    
    #make sure they are in good formate fist and last are [1.0, 0.0]
    child1 = mutation(child1, mutation_intensity)
    child2 = mutation(child2, mutation_intensity)
    
    return child1, child2

In [1]:
def mutation(ctlPts, mut_intens):
    '''this function is to radomize control points locations
        ctlPts: contral points example [[top edge points], [bottom edge points]]
        mut_intens : the probability that the point will be changed
    '''

    for Pts in range(len(ctlPts)):     
        for i in range(len(ctlPts[Pts])):
            if float(random.randint(0,100)) < mut_intens * 100:
                #change x_axis
                #keep the minimum x_axis value higher than or equal to 0
                while round(ctlPts[Pts][i][0], 3) < 0.0:
                    ctlPts[Pts][i][0] += random.uniform(-3, 3) / 100
                #change y_axis
                ctlPts[Pts][i][1] += random.uniform(-1, 1) / 100
    
    #keep the first point and the last point as [1.0, 0.0]
    ctlPts[0][0] = ctlPts[1][-1] = [1.0, 0.0]
    #keep the middle point as 0.0
    ctlPts[0][-1][0] = ctlPts[1][0][0] = 0.0
    #top edge, control points x-axis from large to small
    ctlPts[0] = sorted(ctlPts[0], reverse=True)
    #bottom edge, control points x-axis from small to large
    ctlPts[1] = sorted(ctlPts[1])
    
    #return : [[top edge], [bottom edge]]
    return ctlPts

---
<a id='mining'></a>
## File and Data Mining

---
<a id='eva'></a>
## Heuristic Function

---
<a id='script'></a>
## Script

In [294]:
import psutil as ps

def script(airfoil_loc, airfoil, data_loc, AOT, RE):
    load_script = ['load '+airfoil_loc, 
            airfoil,
            'MDES',
            'FILT',
            'EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 150']       
    #define the output file name (different reynolds number has different file)
    data_file_name = data_loc+'/'+str(RE)+'.txt'
    #check and create file
    constant_set = ['RE ' + str(RE),
                    'VISC ' +str(RE),
                    'PACC', 
                    data_file_name,
                    ' ']
    #analyse different angle of attack
    alpha = ['ALFA '+str(a) for a in AOT]
    init = constant_set + alpha + ['PACC', ' ',]
    load_script += init
    load_script += [' ', ' ', 'QUIT\n']
    return load_script

In [295]:
def execute(airfoil_loc, airfoil, data_loc, angles, rey_range):
    #create a directory for each airfoil
    filedir = data_loc+airfoil
    if not os.path.isdir(filedir):
        os.makedirs(filedir)
    for RE in rey_range:
        #coupling xfoil into python
        if not isfile(filedir+'/'+str(RE)+'.txt'):
            pro = ps.Popen(['xfoil.exe'],
                          stdin=sp.PIPE,
                          stdout=None,
                          stderr=None,
                          encoding='utf8')
            #execute the script
            res = pro.communicate('\n'.join(script(airfoil_loc,airfoil, filedir, angle_range, RE)))

In [296]:
def thickness(airfoil_loc, min_thick):
    '''this is to check if the airfoil have sufficient thickness'''
    boo = False
    
    pro = ps.Popen(['xfoil.exe'],    
                  stdin=sp.PIPE,                  
                  stdout=sp.PIPE,                  
                  stderr=None,                  
                  encoding='utf8')       
    #execute the script   
    res = pro.communicate('\n'.join(['load '+airfoil_loc, 'airfoil\n']))
    
    #get the thickness index of the airfoil
    result_text = list(filter(None, str(res[0]).split(' ')))
    thickness_loc = result_text.index('thickness') + 2
    thickness = float(result_text[thickness_loc])
    print(thickness)
    if thickness > min_thick:
        boo = True
    return boo

---
<a id='run'></a>
## Run

In [297]:
#constants setting
epochs = 1
airfoil_dir = 'airfoil/'
data_dir = 'data/'
#'why do we choose this angle of attack'
angle_range = frange(8, 13, 0.25)
#'how to get this value?'
rey_range = [200000]
n_parents = 5
mut_inten = 0.5
#"control points shouldn't be too many"

In [301]:
#reboot
boostup(data_dir, airfoil_dir)

In [ ]:
from tqdm import tqdm_notebook

parents_names = [] #parents' names
parents_ctlPts = {} #{names: control points}
parents_loc = {} #{names: cordinate locations}
parents_scores = [] # [(airfoil names, scores)]

#set control points
cntPts = ctlPts_setup(n_parents)
#return: {name: cntPts}
parents_ctlPts.update(cntPts)

#run the loop
for n in range(epochs):
    #set generation
    gen = str(n+1)
    #set airfoil and data directory
    data_loc = data_dir+str(gen)+'gen/'
    airfoil_loc = airfoil_dir+str(gen)+'gen/'
    
    #create airfoil dir and data dir
    os.makedirs(data_loc)
    os.makedirs(airfoil_loc)
    
#'foil stage'    
    #generate airfoil cord"inate
    foil_locs, foil_ctlPts, foil_names= generator(airfoil_loc, parents_ctlPts, mut_inten, gen)
    #foil_locs = {names, locations}, foil_ctlPts = {names, control points}, foil_names = [airfoil names]
    
    for Fname in foil_names:
        airfoil_loc = foil_locs[Fname]
        execute(airfoil_loc, Fname, data_loc, angle_range, rey_range)
        #load airfoil from airfoil_loc and use xfoil compute the necessary values nad dump data into data_loc

In [220]:
    
    for airfoil in foil_names:
        airfoil_loc = foil_locs[airfoil]
        execute(airfoil_loc, airfoil, data_loc, ang_range, rey_range)
        #load airfoil from airfoil_loc and use xfoil compute the necessary values nad dump data into data_loc
    
    #get lift coefficient of each angle in each reynolds number
    mata_data = data_mining(data_loc, rey_range)
    #mata_data = [airfoiil name [*rey, [*angle, *Cl]]]
    
    # basically, the winners will be reused in the next generation for comparation see if the next generation got better
    #get the scores in such order than the highers one is the first one
    #we can see, here, the last gen winner will be comparing with this gen data as well
    foil_scores = heuristic(mata_data)

#'choose parents'
    #foil_scores = {airfoil names: scores}
    rank = sort_airfoil(foil_scores, parents_scores)
    #rank = [(airfoil name*, scores*)]
    print(rank)
    
    # choose the number of parents in this model
    parents_scores = rank[:n_parents]
    parents_names = [i for i, _ in parents_scores]
    
    #update the parents' location in dict formate
    #update the new parents airfoil locations
    parents_loc.update({i: airfoil_loc+i+'.txt' for airfoil in parents_names if airfoil not in parents_loc})
    #filt the parents that are no longer good enough
    parents_loc = {airfoil: winners_loc[airfoil] for airfoil in parents_names}
    
    #update the parents' control points
    #update the new parents airfoil control points
    parents_ctlPts.update({airfoil: foil_ctlPts[airfoil] for airfoil in parents_names if airfoil not in parents_ctlPts})
    #filt the parents that are no longer good enough
    parents_ctlPts = {airfoil: parents_ctlPts[airfoil] for airfoil in parents_names}

"    \n    for airfoil in foil_names:\n        airfoil_loc = foil_locs[airfoil]\n        execute(airfoil_loc, airfoil, data_loc, ang_range, rey_range)\n        #load airfoil from airfoil_loc and use xfoil compute the necessary values nad dump data into data_loc\n    \n    #get lift coefficient of each angle in each reynolds number\n    mata_data = data_mining(data_loc, rey_range)\n    #mata_data = [airfoiil name [*rey, [*angle, *Cl]]]\n    \n    # basically, the winners will be reused in the next generation for comparation see if the next generation got better\n    #get the scores in such order than the highers one is the first one\n    #we can see, here, the last gen winner will be comparing with this gen data as well\n    foil_scores = heuristic(mata_data)\n\n#'choose parents'\n    #foil_scores = {airfoil names: scores}\n    rank = sort_airfoil(foil_scores, parents_scores)\n    #rank = [(airfoil name*, scores*)]\n    print(rank)\n    \n    # choose the number of parents in this model

In [249]:
import psutil

ModuleNotFoundError: No module named 'psutil'